In [1]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import matplotlib.pyplot as plt


In [2]:
filename ="root://storage01.lcg.cscs.ch:1096//pnfs/lcg.cscs.ch/cms/trivcat//store/mc/Run3Summer22EENanoAODv12/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/Poisson60KeepRAW_130X_mcRun3_2022_realistic_postEE_v6-v2/50000/fc603037-ef65-4bbf-9cef-934ecec40bbe.root"
filename ="/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/hh4b_12.root"
events = NanoEventsFactory.from_root(filename, schemaclass=NanoAODSchema, entry_stop=20).events()
print("Events read:", len(events))

Events read: 20


In [3]:
# print(events.fields)
# print(events.Electron.fields)
print(events.GenPart)
sum_p4=events.GenPart[:,2]+events.GenPart[:,3]
print(sum_p4.mass)
print(sum_p4.pt)
print(events.GenPart[:,2].pt)
print(events.GenPart[:,3].pt)

[[GenParticle, GenParticle, GenParticle, ... GenParticle, GenParticle, GenParticle]]
[535, 585, 338, 637, 491, 428, 367, 464, ... 413, 420, 403, 376, 501, 326, 385, 418]
[51.2, 29, 84.4, 41, 10.9, 38, 47.4, ... 52.4, 84.7, 52.1, 67.8, 20.2, 10.3, 64.9]
[257, 78.2, 159, 236, 202, 128, 112, 198, ... 178, 35.8, 129, 90.8, 75.8, 85.2, 72.8]
[217, 98, 75.5, 270, 200, 95.8, 144, 176, ... 162, 155, 120, 154, 126, 94, 90.8, 128]


In [4]:
print(events.LHEPart.pdgId)

# Select b-quarks at LHE level
isOutgoing = events.LHEPart.status == 1
print(isOutgoing)

isB = abs(events.LHEPart.pdgId) == 5
# print only events with at least one b-quark
for     i in range(len(isB)):
    if len(isB[isB][i]) > 0:
        print(i)
        print(events.LHEPart.pdgId[i])
        break
bquarks = events.LHEPart[isB & isOutgoing]

[[21, 21, 25, 25, 21], [21, 21, 25, 25, ... 21, 25, 25, 21], [21, 21, 25, 25, 21]]
[[False, False, True, True, True], [False, ... [False, False, True, True, True]]


In [5]:
e=events[2]
print(e.LHEPart.pdgId)
pdg_gen = e.GenPart.pdgId
print(pdg_gen==5)
# select b-quarks at Gen level
isB = abs(pdg_gen) == 5
# select higgs at Gen level
isH = pdg_gen == 25
print(pdg_gen[isB | isH])

b_higgs = e.GenPart[isB | isH]


[21, 21, 25, 25, 21]
[False, False, False, False, False, False, ... False, False, False, False, False]
[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 5, -5, 5, -5, -5, 5, 5, -5]


In [6]:
isHiggs = b_higgs.pdgId == 25
isHard = b_higgs.hasFlags(["fromHardProcess"])
hasTwoChildren = ak.num(b_higgs.childrenIdxG, axis=1) == 2
# print(events.GenPart.childrenIdxG)
# print(ak.num(events.GenPart.childrenIdxG, axis=2))

print(len(e.GenPart.pdgId), e.GenPart.pdgId)
print(len(e.GenPart.childrenIdxG), e.GenPart.childrenIdxG)
print(len(e.GenPart.children.pdgId), e.GenPart.children.pdgId
      )

print(len(b_higgs), b_higgs.pdgId
      )
print(len(b_higgs.childrenIdxG), b_higgs.childrenIdxG
      )
print(len(b_higgs.children.pdgId), b_higgs.children.pdgId
      )

# print(isHiggs)
# print(isHard)
# print(hasTwoChildren)

65 [21, 21, 25, 25, 21, 25, 25, 25, 25, ... -421, 411, 421, 13, -14, 111, 111, 22, 22]
65 [[139, 140, 141], [], [142], [143], [150, 151], ... [], [], [], [200], [201], [], []]
65 [[25, 25, 21], [], [25], [25], [21, 21], [25, ... [], [], [], [], [22], [22], [], []]
20 [25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 5, -5, 5, -5, -5, 5, 5, -5]
20 [[142], [143], [144], [145], [146], [147, ... [166, 167], [158, 159], [], [], [], []]
20 [[25], [25], [25], [25], [25], [25], ... -5], [5, -513], [-5, 523], [], [], [], []]


In [7]:
i = 1

j = i + 1
gen = events.GenPart#[i:j]
isHiggs = gen.pdgId == 25
isHard = gen.hasFlags(["fromHardProcess"])
isLastCopy = gen.hasFlags(["isLastCopy"])
isFirstCopy = gen.hasFlags(["isFirstCopy"])
isB = abs(gen.pdgId) == 5
hasTwoChildren = ak.num(gen.children, axis=2)
print(hasTwoChildren, len(hasTwoChildren))
# for k in range(len(hasTwoChildren[i])):
#     print(
#         k,
#         hasTwoChildren[i, k],
#         gen.pdgId[i, k],
#         gen.childrenIdxG[i, k],
#         gen.children.pdgId[i, k],
#         gen.hasFlags(["isLastCopy"])[i, k],
#         gen.hasFlags(["fromHardProcess"])[i, k],
#     )
print("gen", gen.pdgId[isHiggs & isLastCopy & isHard])
print("gen", gen.childrenIdxG[isHiggs & isLastCopy & isHard])
print("gen", gen.genPartIdxMother[isHiggs & isLastCopy & isHard])
print("gen", gen.pt[isHiggs & isLastCopy & isHard])
print("gen", gen.children.genPartIdxMother[isHiggs & isLastCopy & isHard])

print(gen.childrenIdxG)
print("num", ak.num(gen.childrenIdxG, axis=2)[:, 10:18])

hasTwoChildren = ak.num(gen.children, axis=2) == 2

# print(hasTwoChildren[isHiggs & isLastCopy])
# print(gen[isHiggs & isHard].childrenIdxG)

# print(gen[isHiggs].hasFlags(["isLastCopy"]))
# print(gen[isHiggs].children.pdgId)
# print(gen[isHiggs & hasTwoChildren].children.pdgId)
# print(gen[isHiggs].childrenIdxG)

higgs = gen[isHiggs & isHard & isLastCopy & hasTwoChildren]
# higgs_loose = gen[isHiggs & isHard & isLastCopy]
# num_higgs = ak.num(higgs_loose, axis=1)
# print(num_higgs, len(num_higgs))
# print(gen[num_higgs!=2].pdgId, len(gen[num_higgs==2].pdgId))
print(higgs.childrenIdxG, higgs.children.pdgId)
bquarks = ak.flatten(higgs.children, axis=2)  # [isB & isHard & isFirstCopy]
tot_b = gen[isB & isHard]
b_unflattened = higgs.children
print(b_unflattened.pdgId)
print(b_unflattened.genPartIdxMother)
print(bquarks.pdgId)
print(bquarks.genPartIdxMother)

print("loop")
# loop over the bquarks children until we find the bquarks and then take the last copy
while True:
    b_children = bquarks.children
    print(b_children.pdgId, b_children.genPartIdxMother, len(b_children.pdgId))
    if ak.any(abs(b_children.pdgId) == 5):
        print("found bquark")
        mask_b = abs(b_children.pdgId) == 5
        print(mask_b)
        bquarks = b_children[mask_b]
        print(bquarks.pdgId, bquarks.genPartIdxMother)
        continue

    break

print(bquarks.index, bquarks.pt)
# all_higgs = gen[isHiggs & isHard]
# print("pt", higgs.pt, len(higgs.pt))


# b_pairs = ak.combinations(bquarks, 2)
# print(b_pairs["0"].pdgId)
# print(b_pairs["1"].pdgId)
# print(b_pairs["0"].genPartIdxMother)
# print(b_pairs["1"].genPartIdxMother)


# # given each bquark pair, i want to check if the two bquarks come from the same higgs or not
# same_higgs = (b_pairs["0"].genPartIdxMother == b_pairs["1"].genPartIdxMother)

# print(same_higgs)
# b_pairs = b_pairs[same_higgs]
# print(b_pairs)

# b_pairs_idx = ak.argcombinations(bquarks, 2)
# print(b_pairs_idx )

# unz= ak.unzip(b_pairs_idx)
# print(unz)

# b_pairs_idx=b_pairs_idx[same_higgs]
# print(b_pairs_idx)
# print(ak.unzip(b_pairs_idx))
# b_pair_first =b_pairs_idx[:,0]
# b_pair_second =b_pairs_idx[:,1]

# print(b_pair_first)
# print(b_pair_second)


# print(b_pair_first[:]["0"])

# b11=bquarks[:,b_pair_first[:]["0"]]
# print(b11.pdgId, b11.genPartIdxMother, b11)
# b12=bquarks[:,b_pair_first[:]["1"]]
# print(b12.pdgId, b12.genPartIdxMother, b12)
# b21=bquarks[:,b_pair_second[:]["0"]]
# print(b21.pdgId, b21.genPartIdxMother, b21)
# b22=bquarks[:,b_pair_second[:]["1"]]
# print(b22.pdgId, b22.genPartIdxMother, b22)


# provenance = ak.zeros_like(bquarks.pdgId)
# provenance = ak.to_numpy(provenance)


# for k in [0,1]:
#     print(k)
#     print(b_pairs_idx[:,k]["0"], b_pairs_idx[:,k]["1"])
#     provenance[:,b_pairs_idx[:,k]["0"]] = k+1
#     provenance[:,b_pairs_idx[:,k]["1"]] = k+1

# print(provenance)


# # add a field to bquarks which says if the bquark comes from the first or second higgs
# bquarks=ak.with_field(bquarks, provenance, "provenance")
# print(bquarks.provenance)
# print(bquarks.pdgId)
# print(bquarks.genPartIdxMother)

[[3, 0, 1, 1, 11, 1, 1, 1, 1, 1, 1, 1, 1, ... 0, 0, 0, 3, 1, 0, 2, 0, 0, 0, 0, 0, 0]] 20
gen [[25, 25], [25, 25], [25, 25], [25, 25], ... [25, 25], [25, 25], [25, 25], [25, 25]]
gen [[[17, 18], [19, 20]], [[87, 88], ... [1170, 1171]], [[1228, 1229], [1230, 1231]]]
gen [[13, 14], [9, 10], [11, 12], [11, 12], ... [15, 16], [7, 8], [21, 22], [17, 18]]
gen [[265, 209], [87.2, 108], [162, 72.8], ... [76.2, 91], [84.8, 82.2], [60.1, 154]]
gen [[[15, 15], [16, 16]], [[14, 14], [15, 15, ... 23], [24, 24]], [[19, 19], [20, 20]]]
[[[2, 3, 4], [], [5], [6], [11, 12, 22, ... [1278, 1279], [], [], [], [], [], []]]
num [[1, 1, 1, 1, 1, 2, 2, 4], [1, 4, 1, 1, 2, ... 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]
[[[17, 18], [19, 20]], [[87, 88], ... [1170, 1171]], [[1228, 1229], [1230, 1231]]] [[[5, -5], [5, -5]], [[5, -5], [5, -5]], ... [[5, -5], [5, -5]], [[5, -5], [5, -5]]]
[[[5, -5], [5, -5]], [[5, -5], [5, -5]], ... [[5, -5], [5, -5]], [[5, -5], [5, -5]]]
[[[15, 15], [16, 16]], [[14, 14], [15, 15, ... 2

AttributeError: no field named 'index'

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.3/src/awkward/highlevel.py#L1124)

In [ ]:
# compute invariant mass of each bquark pair with the same provenance
# higgs1=b11+b12
# print(b11.mass)
# print(b12.mass)
# print(b11.px)
# print(b12.px)
# print(b11.py)
# print(b12.py)
# print(b11.pz)
# print(b12.pz)
# print(b11.energy)
# print(b12.energy)
# print(higgs1)
# print(higgs1.mass)


In [ ]:

minDR=0.4
# Compute deltaR(b, jet) and save the nearest jet (deltaR matching)
deltaR = ak.flatten(bquarks.metric_table(events.Jet), axis=2)
# keeping only the pairs with a deltaR min
maskDR = deltaR<  minDR
deltaRcut = deltaR[maskDR]
idx_pairs_sorted = ak.argsort(deltaRcut, axis=1)
pairs = ak.argcartesian([bquarks, events.Jet])[maskDR]
pairs_sorted = pairs[idx_pairs_sorted]
idx_bquarks, idx_JetGood = ak.unzip(pairs_sorted)

use the last b quark for the matching

In [11]:
j=0
filename ="root://storage01.lcg.cscs.ch:1096//pnfs/lcg.cscs.ch/cms/trivcat//store/mc/Run3Summer22EENanoAODv12/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/Poisson60KeepRAW_130X_mcRun3_2022_realistic_postEE_v6-v2/50000/fc603037-ef65-4bbf-9cef-934ecec40bbe.root"
filename ="/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/hh4b_12.root"
events = NanoEventsFactory.from_root(filename, schemaclass=NanoAODSchema, entry_start=j, entry_stop=j+20).events()
print("Events read:", len(events))

Events read: 20


In [12]:
# add column with the index
events.GenPart=ak.with_field(events.GenPart, ak.local_index(events.GenPart, axis=1), "index")
print(events.GenPart.index[0])
print(events.GenPart.index[1])
# get last bquark copy
isB = abs(events.GenPart.pdgId) == 5
isLast = events.GenPart.hasFlags(["isLastCopy"])
isHard = events.GenPart.hasFlags(["fromHardProcess"])
bquarks_last = events.GenPart[isB & isLast & isHard]
# print(bquarks_last.pdgId, bquarks_last.genPartIdxMother)
b_quarks = bquarks_last
while True:
    print("\nloop")
    b_mother = events.GenPart[b_quarks.genPartIdxMother]
    mask_mother = (abs(b_mother.pdgId) == 5) | ((b_mother.pdgId) == 25)

    bquarks_last = bquarks_last[mask_mother]
    b_quarks = b_quarks[mask_mother]
    b_mother = b_mother[mask_mother]
    print("old: ", "pdg", b_quarks.pdgId, "mother_idx",b_quarks.genPartIdxMother, "pt", b_quarks.pt, "index", b_quarks.index)
    print("mother: ", "pdg", b_mother.pdgId, "mother_idx",b_mother.genPartIdxMother, "pt", b_mother.pt)
    # break
    b_quarks = ak.where(abs(b_mother.pdgId) == 5, b_mother, b_quarks)
    print("new: ", "pdg", b_quarks.pdgId, "mother_idx",b_quarks.genPartIdxMother, "pt", b_quarks.pt)

    print(ak.any(abs(b_mother.pdgId) != 25, axis=1))
    if ak.all(abs(b_mother.pdgId) == 25):
        break

print(b_quarks.pdgId, b_quarks.genPartIdxMother, b_quarks.pt)
print(b_quarks.genPartIdxMother, len(b_quarks.genPartIdxMother))
print(b_quarks.index, len(b_quarks.index))
print(bquarks_last.index, len(bquarks_last.index))


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ... 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ... 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]

loop
old:  pdg [[5, -5, 5, -5], [-5, 5, 5, -5], [-5, 5, ... 5, -5], [5, -5, 5, -5], [5, -5, 5, -5]] mother_idx [[17, 18, 19, 20], [17, 16, 20, 19], [20, ... [23, 23, 28, 29], [21, 22, 23, 24]] pt [[116, 124, 186, 13.8], [87.2, 35.4, 103, ... 47.2, 65.8], [31.9, 24.1, 160, 34]] index [[33, 37, 40, 42], [21, 27, 32, 33], [21, ... [26, 27, 42, 43], [33, 34, 37, 38]]
mother:  pdg [[5, -5, 5, -5], [-5, 5, 5, -5], [-5, 5, ... 5, -5], [25, 25, 5, -5], [5, -5, 5, -5]] mother_idx [[15, 15, 16, 16], [14, 14, 18, 15], [16, ... [21, 21, 24, 24], [19, 19, 20, 20]] pt [[128, 164, 218, 21.7], [93, 44, 103, 31.5, ... 84.8, 49.5, 83], [33.2, 34, 164, 34]]
new:  pdg [[5, -5, 5, -5], [-5, 5, 5, -5], [-5, 5, ... 5, -5], [5, -5, 5, -5], [5, -5, 5, -5]] mother_idx [[15, 15, 16, 16], [14, 14, 18, 15], [16, ... [23, 23, 24, 24], [19, 19, 20, 20]

In [14]:
print(bquarks_last.index, len(bquarks_last.index))


[[33, 37, 40, 42], [21, 27, 32, 33], [21, ... [26, 27, 42, 43], [33, 34, 37, 38]] 20


In [13]:
# order higgs pt
higgs = events.GenPart[
            (events.GenPart.pdgId == 25)
            & events.GenPart.hasFlags(["fromHardProcess"])
            & events.GenPart.hasFlags(["isLastCopy"])
        ]
higgs = higgs[ak.num(higgs.childrenIdxG, axis=2) == 2]
print(higgs.pt, higgs.childrenIdxG, higgs.children.pdgId, higgs.children.pt)
print(ak.argsort(higgs.pt,ascending=False))
print(higgs.index)
higgs=higgs[ak.argsort(higgs.pt,ascending=False)]
print(higgs.pt, higgs.childrenIdxG, higgs.children.pdgId, higgs.children.pt)
print(higgs.index)

provenance = ak.where(b_quarks.genPartIdxMother == higgs.index[:,0], 1, 2)
print(provenance)


[[265, 209], [87.2, 108], [162, 72.8], ... [76.2, 91], [84.8, 82.2], [60.1, 154]] [[[17, 18], [19, 20]], [[87, 88], ... [1170, 1171]], [[1228, 1229], [1230, 1231]]] [[[5, -5], [5, -5]], [[5, -5], [5, -5]], ... [[5, -5], [5, -5]], [[5, -5], [5, -5]]] [[[128, 164], [218, 21.7]], [[44, 93], ... [49.5, 83]], [[33.2, 34], [164, 34]]]
[[0, 1], [1, 0], [0, 1], [1, 0], [0, 1], ... [1, 0], [1, 0], [1, 0], [0, 1], [1, 0]]
[[15, 16], [14, 15], [15, 16], [13, 14], ... [17, 18], [10, 11], [23, 24], [19, 20]]
[[265, 209], [108, 87.2], [162, 72.8], ... [91, 76.2], [84.8, 82.2], [154, 60.1]] [[[17, 18], [19, 20]], [[89, 90], ... [1170, 1171]], [[1230, 1231], [1228, 1229]]] [[[5, -5], [5, -5]], [[5, -5], [5, -5]], ... [[5, -5], [5, -5]], [[5, -5], [5, -5]]] [[[128, 164], [218, 21.7]], [[105, 31.5], ... [49.5, 83]], [[164, 34], [33.2, 34]]]
[[15, 16], [15, 14], [15, 16], [14, 13], ... [18, 17], [11, 10], [23, 24], [20, 19]]
[[1, 1, 2, 2], [2, 2, 1, 1], [2, 2, 1, 1, ... 2, 1, 1], [1, 1, 2, 2], [2, 2, 1, 